In [16]:
import os
import cv2
import torch
import numpy as np
from facenet_pytorch import MTCNN
from tqdm import tqdm

In [17]:
# Setup MTCNN
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(image_size=48, margin=0, device=device)

In [28]:
# Define input and output folders
base_input_dir = './FER-2013'  # Change this to your dataset root
output_dir = 'processed_faces'
os.makedirs(output_dir, exist_ok=True)

In [29]:
# Combine both train and test
splits = ['train', 'test']
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [30]:
# Stats counters
stats = {emotion: {
    'total': 0,
    'none_image': 0,
    'no_face': 0,
    'success': 0
} for emotion in emotions}

for split in splits:
    for emotion in emotions:
        input_folder = os.path.join(base_input_dir, split, emotion)
        output_subfolder = os.path.join(output_dir, emotion)
        os.makedirs(output_subfolder, exist_ok=True)

        for img_name in tqdm(os.listdir(input_folder), desc=f"{split}/{emotion}"):
            stats[emotion]['total'] += 1
            img_path = os.path.join(input_folder, img_name)
            image = cv2.imread(img_path)

            if image is None:
                stats[emotion]['none_image'] += 1
                continue

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            face = mtcnn(image_rgb)

            if face is None:
                stats[emotion]['no_face'] += 1
                continue

            face_np = face.permute(1, 2, 0).numpy()
            face_gray = cv2.cvtColor((face_np * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
            face_normalized = face_gray / 255.0

            save_path = os.path.join(output_subfolder, f"{split}_{img_name}")
            cv2.imwrite(save_path, (face_normalized * 255).astype(np.uint8))
            stats[emotion]['success'] += 1

test/surprise: 100%|██████████| 831/831 [00:04<00:00, 191.09it/s]


In [35]:
# Summary Report
print("Summary Report")
print(f"{'Emotion':<10} | {'Total':<6} | {'Success':<8} | {'Image None':<11} | {'No Face':<8} | {'Success %':<10}")
print("-" * 65)

for emotion in emotions:
    total = stats[emotion]['total']
    success = stats[emotion]['success']
    none_image = stats[emotion]['none_image']
    no_face = stats[emotion]['no_face']
    percent = (success / total) * 100 if total > 0 else 0

    print(f"{emotion:<10} | {total:<6} | {success:<8} | {none_image:<11} | {no_face:<8} | {percent:>9.2f}%")

Summary Report
Emotion    | Total  | Success  | Image None  | No Face  | Success % 
-----------------------------------------------------------------
angry      | 4953   | 4186     | 0           | 767      |     84.51%
disgust    | 547    | 432      | 0           | 115      |     78.98%
fear       | 5121   | 4071     | 0           | 1050     |     79.50%
happy      | 8989   | 7789     | 0           | 1200     |     86.65%
neutral    | 6198   | 5541     | 0           | 657      |     89.40%
sad        | 6077   | 4698     | 0           | 1379     |     77.31%
surprise   | 4002   | 3442     | 0           | 560      |     86.01%
